## Script to monitor replication of source and target cluster

In [ ]:
import requests, json, operator, getpass

#### Configure your credentials here:

In [ ]:
source_auth = ( 'USER', 'PW' )
source_url = 'http://ACCOUNT.cloudant.com'

target_auth = ( 'USER', 'PW' )
target_url = 'https://ACCOUNT.cloudant.com'

target_adm_auth = ( 'USER', 'PW')  # change this to your admin user credentials

#### Fetch the active tasks of source and target

In [ ]:
source_request = requests.get(source_url + '/_active_tasks', auth=source_auth)
target_request = requests.get(target_url + '/_active_tasks', auth=target_auth)



if(source_request.status_code == 200):
    print "Connection to Source: "+str(source_request.status_code)+" (OK)"
    source_active_tasks = source_request.json()
else:
    print "Connection to Source problem: "+str(source_request.status_code)+" (HTTP)"

if(target_request.status_code == 200):
    print "Connection to Target: "+str(target_request.status_code)+" (OK)"
    target_active_tasks = target_request.json()
else:
    print "Connection to Target problem: "+str(target_request.status_code)+" (HTTP)"

#### Extract the dbnames from the source replication URL

In [ ]:
source_active_dbs = {d['source'].split("/")[-2]:d for d in source_active_tasks if d.has_key('source')}
target_active_dbs = {d['source'].split("/")[-2]:d for d in target_active_tasks if d.has_key('source')}

#### Get the DB sizes

In [ ]:
active_db = {}

print "{0: <50} {1:>50} ".format('Source DB', 'TargetDB')
print "{0: <50} {1:>50} ".format('-----------', '-----------')

for db in source_active_dbs:
    if db.startswith('on_') or db.startswith('sm_') or db.startswith('nlt_'): # You can filter your DBs here!
        source_db_info = requests.get(source_url + '/' + db, auth=source_auth).json()
        target_db_info = requests.get(target_url + '/' + db, auth=target_auth).json()
        
        source_db_name = source_db_info.get("db_name")
        target_db_name = target_db_info.get("db_name")
        
        target_changes_pending = ""
        try:
            target_changes_pending = target_active_dbs.get(db_name).get("changes_pending", "")
        except:
            pass
        
        print "{0: <50} {1:>50} ".format(source_db_name, target_db_name)
        
        try:
            active_db[source_db_name] = ( 
                    source_db_info.get("doc_count", ""),  
                    target_db_info.get("doc_count", ""),
                    target_db_info.get("other", "").get("data_size", ""),
                    target_changes_pending
                    )
        except:
            pass
    else:
        print db+" not included"

#### Sort the list of active_dbs in order of data_size

In [ ]:
sorted_active_dbs = sorted(active_db.items(), key=operator.itemgetter(1))
    
print "{0: <20} {1:>20} {2:>15} {3:>10} {4:>10} {5:>10} {6:>15}".format('DB name', 'Source count', 'Target count', 'Diff', 'Pct',  'Size Gb',  'Chgs Pend[1]')
print "{0: <20} {1:>20} {2:>15} {3:>10} {4:>10} {5:>10} {6:>15}".format('----------------', '-----------', '--------------', '----', '---',  '-------',  '------------')

def num(s):
    try:
        return int(s)
    except ValueError:
        return 0

import math

total_sze = 0
    
for db in sorted_active_dbs:
    db_name = db[0]
    source_cnt  = num(db[1][0])
    target_cnt  = num(db[1][1])
    target_sze  = num(db[1][2]) / 1024 / 1024 / 1024
    target_pen  = num(db[1][3])
    
    total_sze = total_sze + target_sze
    
    diff    = source_cnt - target_cnt
    pct     = 100 * (target_cnt / float(source_cnt))
    shards  = int(math.ceil((source_cnt * 1.5) / 10000000)) # 1.5 is expected growth

    print "{0: <20} {1:>20} {2:>15} {3:>10} {4:>10} {5:>10} {6:>15}".format(db_name, source_cnt, target_cnt, diff, int(pct), target_sze, target_pen )
        
print "{0: <20} {1:>20} {2:>15} {3:>10} {4:>10} {5:>10}".format('', '', '', '', '',  '-------')
print "{0: <20} {1:>20} {2:>15} {3:>10} {4:>10} {5:>10}".format('', '', '', '', '',  str(total_sze)+" GB")
        
print "[1] changes_pending is from target: {}/_active_tasks".format(target_url)